In [64]:
import argparse
import pickle
from collections import defaultdict
from collections import Counter
#from ipdb import set_trace
#from negative_samples import negative_sampler
import numpy as np
import os
#from sma_toolkit import embeddings as emb_utils 
#import streaming_pickle as stPickle
import time
import random
import tensorflow as tf
np.set_printoptions(threshold=np.nan)
MIN_DOC_LEN=4

Create Negative Samples that are user specific and then using negative samples and the previous user word ids create user specific train data. create_user_train function is similar to the batch process of skigram model of word2vec.

In [188]:
embed = "word2vec_nce.model"
train_file = "user_text_sample.txt"
output_pkl = "train_embeddings.pkl"
vocabulary_size = 100000
min_word_freq = 5
seed = 42
neg_samples = 10
output = "user_train_data.pkl"

In [189]:
def load_embeddings(filename,wordDict,embedding_size):
    dictionary, steps, word_embeds = pickle.load(open(filename, 'rb'))
    embedding_array = np.zeros((len(wordDict), embedding_size))
    knownWords = list(wordDict.keys())
    #print(knownWords)
    foundEmbed = 0
    for i in range(len(embedding_array)):
        index = -1
        
        w = knownWords[i]
        if w in dictionary:
            index = dictionary[w]
        elif w.lower() in dictionary:
            index = dictionary[w.lower()]
        if index >= 0:
            foundEmbed += 1
            embedding_array[i] = word_embeds[index]
        else:
            embedding_array[i] = np.random.rand(embedding_size) * 0.02 - 0.01
    #print "Found embeddings: ", foundEmbed, "/", len(knownWords)
    #print "embedding_array shape",embedding_array.shape

    return embedding_array
    

In [190]:
def get_neg_samples(user_dict, wc):
    neg_dict = defaultdict(list)
    #sample = np.random.choice(vocabulary_size, num_sampled, p=unigram_prob, replace=False)
   
    for user, message in user_dict.items():
        user_wrd = set(message)
        word_corpus = set(wc.keys())
        diff = word_corpus - user_wrd        
        neg_dict[user] = random.sample(diff, 15)
    return neg_dict

In [194]:
def create_embeds():
    
    
    
    t0 = time.time()
    word_counter = Counter()
    n_docs=0
    embedding_size = 128
    user_dict = defaultdict(list)
    c = 0
    with open(train_file,"r") as fid:
        for line in fid:
#             print("len", len(line))
            if len(line) > 1:
                message = line.split()
                user_dict[message[0]].extend(message[1:])
                word_counter.update(message[1:])
                n_docs += 1
    
    #keep only words that occur at least min_word_freq times
    wc = {w:c for w,c in word_counter.items() if c > min_word_freq} 
    #finding unigram probability
    unigram_cnt = [c for w, c in wc.items()]
    total = sum(unigram_cnt)
    unigram_prob = [c*1.0/total for c in unigram_cnt]
        
    tw = sorted(wc.items(), key=lambda x:x[1],reverse=True)
    
    top_words = {w[0]:i for i,w in enumerate(tw[:vocabulary_size])}
    print(type(top_words))
    wrd2idx = {w:i for i,w in enumerate(top_words.keys())}
    
    #full_E, full_wrd2idx = emb_utils.read_embeddings(args.emb,top_words)
    
    embed_matrix = load_embeddings(embed,top_words,embedding_size)
    embeddings = tf.Variable(embed_matrix, dtype=tf.float32)
    print(embeddings)
    pickle.dump([embed_matrix,unigram_prob,wrd2idx,word_counter,len(user_dict.keys())], open(output_pkl, 'wb'))
    
    return user_dict,wc,wrd2idx
    

In [195]:
def create_user_train(user_dict, wc, wrd2idx):
    negative_samples = get_neg_samples(user_dict, wc)
    #print(negative_samples)

    prev_user, prev_user_data, prev_ctxscores, prev_neg_samples  = None, [], [], []
    full_train = []
#     f_train = open(output,"wb")
    rng = np.random.RandomState(seed)
    with open(train_file, "r") as fid:
        for j, line in enumerate(fid):
            if len(line) <= 1:
                continue

            message = line.split()
            user_id = message[0]

            #convert to indices
            msg_idx = [wrd2idx[w] for w in message if w in wrd2idx]
            neg_idx = [wrd2idx[w] for w in negative_samples[user_id] if w in wrd2idx]
            
            if prev_user == None:  # first user
                prev_user = user_id
            elif user_id != prev_user or j == n_docs - 1: # this user_id is seen for the first time
                
                # get numbers in range [0, len(prev_user_data))
                shuf_idx = np.arange(len(prev_user_data))
                # shuffle numbers
                rng.shuffle(shuf_idx)

                # fill these lists with the same data in a different order
                prev_user_data = [prev_user_data[i] for i in shuf_idx]
                prev_neg_samples = [prev_neg_samples[i] for i in shuf_idx]
                
                # 90-10 train-test split
                split = int(len(prev_user_data)*.9)
                train = prev_user_data[:split]
                test  = prev_user_data[split:]

                neg_samples = prev_neg_samples[:split]
                # each training instance consists of:
                # [user_name, train docs, test docs, negative samples]
                full_train.append([prev_user, train, test, neg_samples])
                prev_user_data = []
                prev_neg_samples = []

            prev_user = user_id
            prev_user_data.append(msg_idx)
            prev_neg_samples.append(neg_idx)

    pickle.dump(full_train, open(output, 'wb'))
   

In [196]:
user_dict, wc, wrd2idx = create_embeds()
create_user_train(user_dict,wc,wrd2idx)

<class 'dict'>
<tf.Variable 'Variable_55:0' shape=(78, 128) dtype=float32_ref>
